In [5]:
import sys
import os
import pandas as pd
import numpy as np
from datatable import dt,count,mean,min,max,by,f,sort
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import datatable
import matplotlib
import plotly

## check the version of main packages. 

In [9]:
print("pd.__version__:",pd.__version__)
print("np.__version__:",np.__version__)
print("datatable.__version__:",datatable.__version__)
print("matplotlib.__version__:",matplotlib.__version__)
print("plotly.__version__:",plotly.__version__)

pd.__version__: 1.3.4
np.__version__: 1.20.3
datatable.__version__: 1.0.0
matplotlib.__version__: 3.4.3
plotly.__version__: 5.6.0


# some business topic before analaysing the actual data
  imaging you are a host who will decide to put some houses into the tokyo AirBNB market. i’d better to understand how the market situation. for example, which location, which area, what kind of rom etc will be the popular to the local customers and for Foreign customers. with these kinds of questions, to check is there any answer information included in this tokyo AirBNB dataset via data analysis.
1) how many host are avaliable in the tokyo airBNB market japan?
2) is there any relationship between price and the datetime?
3) which area have the most listing count and may be the most popular for customers?
4) what kinds of host are listed in the AirBNB tokyo market and how many lists per host?
5) how the price for different room type?
6) which area have the most review count number?

## load the AirBNB tokyo dataset
 1) browse the head data for each type dataset
 2) check the data type for each column included in each dataset

In [101]:
# load the dataset
tokyo_cal_file="../Airbnb_open_data/01_tokyo/calendar.csv"
#tokyo_cal_dt=dt.fread(tokyo_cal_file)
tokyo_cal_pd=pd.read_csv(tokyo_cal_file)

tokyo_listings_file="../Airbnb_open_data/01_tokyo/listings.csv"
#tokyo_listings_dt=dt.fread(tokyo_listings_file)
tokyo_listings_pd=pd.read_csv(tokyo_listings_file)

tokyo_reviews_file="../Airbnb_open_data/01_tokyo/reviews.csv"
#tokyo_reviews_dt=dt.fread(tokyo_reviews_file)
tokyo_reviews_pd=pd.read_csv(tokyo_reviews_file)

In [107]:
type(tokyo_reviews_pd)

pandas.core.frame.DataFrame

In [51]:
tokyo_cal_pd.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,197677,2021-12-28,f,"$11,000.00","$11,000.00",3,365
1,4958750,2021-12-28,f,"$5,000.00","$5,000.00",3,29
2,4958750,2021-12-29,t,"$5,000.00","$5,000.00",3,29
3,4958750,2021-12-30,t,"$5,000.00","$5,000.00",3,29
4,4958750,2021-12-31,t,"$5,000.00","$5,000.00",3,29


In [52]:
tokyo_listings_pd.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,197677,Oshiage Holiday Apartment,964081,Yoshimi & Marek,NaN,Sumida Ku,35.71707,139.82608,Entire home/apt,11000,3,165,2020-03-04,1.32,1,363,0,M130003350
1,776070,Kero-kero house room 1,801494,Kei,NaN,Kita Ku,35.73844,139.76917,Private room,8027,3,228,2020-01-18,2.06,1,257,0,M130000243
2,1196177,Stay with host Cozy private room Senju area,5686404,Yukiko,NaN,Adachi Ku,35.74475,139.79731,Private room,40000,2,95,2020-03-17,0.93,1,88,0,M130007760
3,1249571,Unique stay! New&1min to the subway,6809418,Yume,NaN,Shibuya Ku,35.65111,139.72165,Entire home/apt,35000,4,102,2020-12-27,1.01,1,354,0,M130000957
4,1263000,SHIBUYA 7 MINS AMAZING HOST（for woman only）,6876728,Haruko,NaN,Setagaya Ku,35.60742,139.67616,Private room,8286,3,185,2019-12-31,1.79,1,361,0,M130000057


In [79]:
tokyo_reviews_pd.head()

,listing_id,date
0,197677,2011-09-21
1,197677,2011-10-14
2,197677,2011-11-21
3,197677,2011-11-30
4,197677,2012-01-09


In [102]:
## check the data type for each column included in each dataset
tokyo_cal_pd.dtypes

listing_id         int64
date              object
available         object
price             object
adjusted_price    object
minimum_nights     int64
maximum_nights     int64
dtype: object

In [55]:
tokyo_listings_pd.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group               float64
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
number_of_reviews_ltm               int64
license                            object
dtype: object

In [56]:
tokyo_reviews_pd.dtypes

listing_id     int64
date          object
dtype: object

## data pre-processing
1) change the price data type from string into numberic
2) check is there any duplicated data, and remove the duplicated data
3) check is there any nan data 


In [57]:
tokyo_cal_pd.duplicated().sum()

0

In [58]:
tokyo_listings_pd.duplicated().sum()

0

In [59]:
tokyo_reviews_pd.duplicated().sum()

3965

In [60]:
tokyo_reviews_pd=tokyo_reviews_pd.drop_duplicates()

In [61]:
tokyo_reviews_pd.duplicated().sum()

0

In [103]:
tokyo_cal_pd.isna().sum()

listing_id        0
date              0
available         0
price             0
adjusted_price    0
minimum_nights    0
maximum_nights    0
dtype: int64

In [104]:
tokyo_listings_pd.isna().sum()

id                                    0
name                                  0
host_id                               0
host_name                            10
neighbourhood_group               10314
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                        2082
reviews_per_month                  2082
calculated_host_listings_count        0
availability_365                      0
number_of_reviews_ltm                 0
license                              61
dtype: int64

In [105]:
tokyo_reviews_pd.isna().sum()

listing_id    0
date          0
dtype: int64

In [62]:
# convert the price values from string data into numberic data for calculation in further
tokyo_cal_pd["price"]=tokyo_cal_pd["price"].str.replace(",","")
tokyo_cal_pd["price"]=[ item.split(".")[0] for item in tokyo_cal_pd["price"] ]
tokyo_cal_pd["price"]=tokyo_cal_pd["price"].str.replace("$","")
tokyo_cal_pd["price"] = [int(data) for data in tokyo_cal_pd["price"] ]
tokyo_cal_pd["adjusted_price"]=tokyo_cal_pd["adjusted_price"].str.replace(",","")
tokyo_cal_pd["adjusted_price"]=[ item.split(".")[0] for item in  tokyo_cal_pd["adjusted_price"] ]
tokyo_cal_pd["adjusted_price"]=tokyo_cal_pd["adjusted_price"].str.replace("$","")
tokyo_cal_pd["adjusted_price"]= [ int(data) for data in tokyo_cal_pd["adjusted_price"] ]
#chaneg the listing_id values from numbric data into string data 
tokyo_cal_pd["listing_id"] = [ str(data) for data in tokyo_cal_pd["listing_id"] ]


/var/folders/05/6bsl497d4jn2ftkk7d4xms9r0000gn/T/ipykernel_12039/2152879404.py:3: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/var/folders/05/6bsl497d4jn2ftkk7d4xms9r0000gn/T/ipykernel_12039/2152879404.py:7: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [80]:
#chaneg the listing_id&id values from numbric data into string data 
tokyo_listings_pd["id"] = [ str(data) for data in tokyo_listings_pd["id"] ]
tokyo_reviews_pd["listing_id"] = [ str(data) for data in tokyo_reviews_pd["listing_id"] ]

In [82]:
#check the data type for each column
tokyo_listings_pd.dtypes

id                                 object
name                               object
host_id                             int64
host_name                          object
neighbourhood_group               float64
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
number_of_reviews_ltm               int64
license                            object
dtype: object

In [83]:
#check the data type for each column
tokyo_reviews_pd.dtypes

listing_id    object
date          object
dtype: object

In [63]:
#check the price data again
tokyo_cal_pd.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,197677,2021-12-28,f,11000,11000,3,365
1,4958750,2021-12-28,f,5000,5000,3,29
2,4958750,2021-12-29,t,5000,5000,3,29
3,4958750,2021-12-30,t,5000,5000,3,29
4,4958750,2021-12-31,t,5000,5000,3,29


In [64]:
# starting using datatable lib for data analysis instead of pandas
tokyo_cal_dt=dt.Frame(tokyo_cal_pd)

# 

In [65]:
tokyo_cal_dt.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,197677,2021-12-28,f,11000,11000,3,365
1,4958750,2021-12-28,f,5000,5000,3,29
2,4958750,2021-12-29,t,5000,5000,3,29
3,4958750,2021-12-30,t,5000,5000,3,29
4,4958750,2021-12-31,t,5000,5000,3,29
5,4958750,2022-01-01,t,5000,5000,3,29
6,4958750,2022-01-02,t,5000,5000,3,29
7,4958750,2022-01-03,t,5000,5000,3,29
8,4958750,2022-01-04,t,5000,5000,3,29


In [66]:
tokyo_cal_dt_t=tokyo_cal_dt[f.available=="t",:]
tokyo_cal_dt_t.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,4958750,2021-12-29,t,5000,5000,3,29
1,4958750,2021-12-30,t,5000,5000,3,29
2,4958750,2021-12-31,t,5000,5000,3,29
3,4958750,2022-01-01,t,5000,5000,3,29
4,4958750,2022-01-02,t,5000,5000,3,29
5,4958750,2022-01-03,t,5000,5000,3,29
6,4958750,2022-01-04,t,5000,5000,3,29
7,4958750,2022-01-05,t,5000,5000,3,29
8,4958750,2022-01-06,t,5000,5000,3,29


In [67]:
avaliable_counts=tokyo_cal_dt_t[:,count(),by(f.date)]
#avaliable_counts_sorted=avaliable_counts[:,:,sort(f.data)]
avaliable_counts.head()

,date,count
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,2021-12-28,2217
1,2021-12-29,2983
2,2021-12-30,2961
3,2021-12-31,2053
4,2022-01-01,3925
5,2022-01-02,4552
6,2022-01-03,5129
7,2022-01-04,5491
8,2022-01-05,5676


In [106]:
type(avaliable_counts)

datatable.Frame

In [129]:
def plot_data(plot_data,x_col_name,y_col_name):
    """
    plot the data by using plotly go.Figure function

    Args:
        plot_data (dataframe): the data will be plotted
        x_col_name (str): the column name which will be used for plotting the data in x label
        y_col_name (str): the column name which will be used for plotting the data in y label
        
    Returns: None
    """
    if isinstance(plot_data,datatable.Frame):
        if (x_col_name in plot_data.names) and (y_col_name in plot_data.names):
            fig = go.Figure(data=go.Scatter(x=np.hstack(np.asarray(plot_data[x_col_name])),y=np.hstack(np.asarray(plot_data[y_col_name])), mode='lines'))
        else:
            print("ERROR for plotting the data")
    if isinstance(plot_data,pd.core.frame.DataFrame):
        if (x_col_name in plot_data.columns) and (y_col_name in plot_data.columns):
            fig = go.Figure(data=go.Scatter(x=np.asarray(plot_data[x_col_name].values),y=np.asarray(plot_data[y_col_name].values), mode='lines'))
        else:
            print("ERROR for plotting the data")
    fig.show()

In [130]:
isinstance(avaliable_counts,pd.core.frame.DataFrame)

False

In [131]:
plot_data(avaliable_counts,"date","count")

In [132]:
available_per_list=tokyo_cal_dt_t[:,count(),by(f.listing_id)]
available_per_list_sorted=available_per_list[:,:,sort(-f.count)]
available_per_list_sorted.head()

,listing_id,count
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,10505175,365
1,12767276,365
2,13566615,365
3,14523283,365
4,15554671,365
5,1571298,365
6,18951293,365
7,18951757,365
8,19618754,365


In [133]:
type(available_per_list_sorted)

datatable.Frame

In [134]:
plot_data(available_per_list_sorted,"listing_id","count")

In [71]:
ave_price_per_list=tokyo_cal_dt_t[:,mean(f.adjusted_price),by(f.listing_id)]
ave_price_per_list.head()

,listing_id,adjusted_price
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,10002680,6400
1,10024434,5811.19
2,10041494,10872.1
3,10061035,3500
4,10061086,26000
5,10063063,10000
6,10091405,5008.33
7,10094822,18729.8
8,10110295,7345.47


In [135]:
plot_data(ave_price_per_list,"listing_id","adjusted_price")

In [72]:
mean_price_per_date=tokyo_cal_dt_t[:,mean(f.adjusted_price),by(f.date)]
mean_price_per_date.head()

,date,adjusted_price
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,2021-12-28,14281.1
1,2021-12-29,15298
2,2021-12-30,16605.8
3,2021-12-31,18198.3
4,2022-01-01,18041.1
5,2022-01-02,15606
6,2022-01-03,14613.8
7,2022-01-04,13210.1
8,2022-01-05,12885.8


In [136]:
plot_data(mean_price_per_date,"date","adjusted_price")

In [84]:
tokyo_listings_dt=dt.Frame(tokyo_listings_pd)
tokyo_listings_dt.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,…,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,197677,Oshiage Holiday Apartment,964081,Yoshimi & Marek,NA,Sumida Ku,35.7171,139.826,Entire home/apt,11000,…,1.32,1,363,0,M130003350
1,776070,Kero-kero house room 1,801494,Kei,NA,Kita Ku,35.7384,139.769,Private room,8027,…,2.06,1,257,0,M130000243
2,1196177,Stay with host Cozy private room Senju area,5686404,Yukiko,NA,Adachi Ku,35.7448,139.797,Private room,40000,…,0.93,1,88,0,M130007760
3,1249571,Unique stay! New&1min to the subway,6809418,Yume,NA,Shibuya Ku,35.6511,139.722,Entire home/apt,35000,…,1.01,1,354,0,M130000957
4,1263000,SHIBUYA 7 MINS AMAZING HOST（for woman only）,6876728,Haruko,NA,Setagaya Ku,35.6074,139.676,Private room,8286,…,1.79,1,361,0,M130000057
5,1298200,Pine House 2 Bed Room + 2 Toilets,6606987,K & N,NA,Katsushika Ku,35.7399,139.852,Entire home/apt,13500,…,3.27,2,145,0,M130009943
6,1306664,3rd floor penthouse @ heart of TKO,7103955,Masaharu,NA,Shinjuku Ku,35.7026,139.741,Entire home/apt,34600,…,1.63,2,130,1,M130003681
7,1394124,Solar power/Car space/near to park,5710367,David,NA,Meguro Ku,35.6204,139.7,Entire home/apt,35000,…,2.21,1,43,3,Hotels and Inns Business Act | 目黒区保健所 | 目健衛環き第88号
8,1394409,Room in Mejiro/Ikebukuro (2 of 3),7537357,Yayoi,NA,Toshima Ku,35.7242,139.702,Private room,4000,…,2.23,2,87,0,M130000166


In [85]:
neigh_count=tokyo_listings_dt[:,count(),by(f.neighbourhood)]
neigh_count_sorted=neigh_count[:,:,sort(-f.count)]

In [137]:
plot_data(neigh_count_sorted,"neighbourhood","count")

In [87]:
ave_price_per_type=tokyo_listings_dt[:,mean(f.price),by(f.room_type)]
ave_price_per_type

,room_type,price
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,Entire home/apt,18426.9
1,Hotel room,15045
2,Private room,16125
3,Shared room,4915.67


In [119]:
tokyo_listings_dt[:,count(),by(f.room_type)]

,room_type,count
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,Entire home/apt,6922
1,Hotel room,397
2,Private room,2619
3,Shared room,376


In [98]:
review_count_per_list_ranking=tokyo_listings_dt[:,count(),by(f.number_of_reviews)]
review_count_per_list_ranking.head()

,number_of_reviews,count
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,2082
1,1,846
2,2,547
3,3,412
4,4,311
5,5,270
6,6,249
7,7,222
8,8,192


In [138]:
plot_data(review_count_per_list_ranking,"number_of_reviews","count")

In [140]:
tokyo_listings_dt.names

('id',
 'name',
 'host_id',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'latitude',
 'longitude',
 'room_type',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'last_review',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365',
 'number_of_reviews_ltm',
 'license')

In [141]:
mean_reviws=tokyo_listings_dt[:,mean(f.reviews_per_month),by(f.neighbourhood)]
mean_reviws_sorted=mean_reviws[:,:,sort(-f.reviews_per_month)]
mean_reviws_sorted

,neighbourhood,reviews_per_month
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,Ome Shi,2.42
1,Meguro Ku,1.67404
2,Komae Shi,1.495
3,Shibuya Ku,1.46501
4,Okutama Machi,1.44333
5,Koganei Shi,1.36
6,Shinagawa Ku,1.34681
7,Bunkyo Ku,1.34085
8,Nakano Ku,1.27553


In [148]:
mean_price_per_neigh=tokyo_listings_dt[:,mean(f.price),by(f.neighbourhood)]
mean_price_per_neigh_sorted=mean_price_per_neigh[:,:,sort(f.price)]
mean_price_per_neigh_sorted

,neighbourhood,price
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,Musashimurayama Shi,3666.67
1,Akishima Shi,3880.5
2,Kunitachi Shi,4159.14
3,Higashikurume Shi,4300
4,Akiruno Shi,5284
5,Machida Shi,6002.12
6,Tama Shi,6077.56
7,Hamura Shi,6279
8,Fuchu Shi,6677.7


In [149]:
plot_data(mean_price_per_neigh_sorted,"neighbourhood","price")

In [29]:
tokyo_listings_dt[:,count(),by(f.availability_365)]

,availability_365,count
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,1318
1,1,36
2,2,26
3,3,22
4,4,20
5,5,12
6,6,12
7,7,25
8,8,22


In [147]:

tokyo_list_count=tokyo_listings_dt[:,count(),by(f["host_id"])]
tokyo_list_count_sorted=tokyo_list_count[:,:,sort(-f["count"])]
tokyo_list_count_sorted

,host_id,count
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,258668827,98
1,261951920,63
2,321365764,60
3,28449418,50
4,197405131,49
5,71735650,45
6,30324135,42
7,4838043,41
8,94302731,40


In [143]:
len(set(tokyo_listings_dt["host_id"].to_list()[0]))

2554

In [151]:
np.sum(tokyo_list_count_sorted["count"].to_list()[0])

10314

In [150]:
listing_count_list=list(set(tokyo_list_count_sorted["count"].to_list()[0]))
for item in listing_count_list:
    print("{} hosts have {} listings: about {}% :".format(tokyo_list_count_sorted[f["count"]==item,:].shape[0],item,tokyo_list_count_sorted[f["count"]==item,:].shape[0]/tokyo_list_count_sorted.shape[0]*100))

1110 hosts have 1 listings: about 43.46123727486296% :
403 hosts have 2 listings: about 15.779169929522318% :
232 hosts have 3 listings: about 9.08379013312451% :
178 hosts have 4 listings: about 6.9694596711041505% :
105 hosts have 5 listings: about 4.111198120595145% :
106 hosts have 6 listings: about 4.150352388410337% :
64 hosts have 7 listings: about 2.505873140172279% :
61 hosts have 8 listings: about 2.3884103367267033% :
41 hosts have 9 listings: about 1.6053249804228662% :
31 hosts have 10 listings: about 1.2137823022709475% :
32 hosts have 11 listings: about 1.2529365700861395% :
27 hosts have 12 listings: about 1.05716523101018% :
19 hosts have 13 listings: about 0.7439310884886452% :
23 hosts have 14 listings: about 0.9005481597494126% :
8 hosts have 15 listings: about 0.31323414252153486% :
14 hosts have 16 listings: about 0.548159749412686% :
13 hosts have 17 listings: about 0.5090054815974941% :
8 hosts have 18 listings: about 0.31323414252153486% :
9 hosts have 19 listi